### Imports, Globals, and Configuration

In [10]:
# Imports and globals
import pandas as pd
import numpy as np
import json
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

CONFIG_PATH="../config/config.json"
# YEARS=["2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"]

### Define Functions

In [13]:
# Define functions
# Load configuration
with open(CONFIG_PATH) as fp:
    config = json.load(fp)

def get_df(file_path, year=None):
    '''
    Returns a dataframe for a specific year.
    '''
    return pd.read_csv(file_path)

KeyError: 'data_path'

### Load DataFrame Dictionary (heavy compute for international files)

In [14]:
df = get_df(config["domestic_data_path"])

### Data Preprocessing

In [18]:
# X = df.drop(columns=['class'])
print(df["arr_cancelled"])

def preprocess_data(X):
    # convert categorical data to numerical
    le = LabelEncoder()
    for column in X.columns:
        X[column] = le.fit_transform(X[column])

    

# Get scaler
scaler = StandardScaler()



KeyError: "['class'] not found in axis"